# Hyperparameter Tuning using HyperDrive

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [ ]:
import azureml.core
from azureml.core import Workspace, Experiment, ScriptRunConfig
from azureml.core.dataset import Dataset
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive import HyperDriveConfig, PrimaryMetricGoal, RandomParameterSampling, choice, BanditPolicy
from azureml.widgets import RunDetails
from azureml.core.model import Model
from azureml.core.environment import Environment
from azureml.core.model import InferenceConfig
from azureml.core.webservice import AciWebservice

from sklearn.datasets import load_iris
from sklearn.ensemble import RandomForestClassifier
import os
import pandas as pd
import requests

## Dataset

TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [ ]:
ws = Workspace.from_config()
experiment_name = 'exp_iris_hyperdrive'

experiment=Experiment(ws, experiment_name)

In [ ]:
iris = load_iris()

# Combine features and target into a single DataFrame
iris_df = pd.DataFrame(data=iris.data, columns=iris.feature_names)
iris_df['target'] = iris.target

# Register the dataset
# datastore = ws.get_default_datastore()
# iris_dataset = Dataset.Tabular.register_pandas_dataframe(
#     name='iris_dataset',
#     dataframe=iris_df,
#     target=(datastore, 'iris_dataset')
# )

## Hyperdrive Configuration

TODO: Explain the model you are using and the reason for chosing the different hyperparameters, termination policy and config settings.

In [ ]:
# TODO: Create an early termination policy. This is not required if you are using Bayesian sampling.
early_termination_policy = BanditPolicy(slack_factor=0.1, evaluation_interval=1, delay_evaluation=5)

#TODO: Create the different params that you will be using during training
param_sampling = RandomParameterSampling({
    "--n_estimators": choice(50, 100, 150, 200),
    "--max_depth": choice(5, 10, 15, 20)
})

#TODO: Create your estimator and hyperdrive config
env = Environment.from_conda_specification(name='sklearn_env', file_path='environment.yml')
script_config = ScriptRunConfig(source_directory=os.getcwd(), 
                                script='train.py', 
                                compute_target='notebook260045',
                                environment=env)

hyperdrive_run_config = HyperDriveConfig(run_config=script_config,
                                         hyperparameter_sampling=param_sampling,
                                         policy=early_termination_policy,
                                         primary_metric_name="Accuracy",
                                         primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
                                         max_total_runs=20,
                                         max_concurrent_runs=4)

In [ ]:
#TODO: Submit your experiment
hyperdrive_run = experiment.submit(hyperdrive_run_config)

## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [ ]:
RunDetails(hyperdrive_run).show()

## Best Model

TODO: In the cell below, get the best model from the hyperdrive experiments and display all the properties of the model.

In [ ]:
best_run = hyperdrive_run.get_best_run_by_primary_metric()
best_run_metrics = best_run.get_metrics()
parameter_values = best_run.get_details()['runDefinition']['arguments']

print('Best Run Id: ', best_run.id)
print('\n Accuracy:', best_run_metrics['Accuracy'])
print('\n n_estimators:', parameter_values[1])
print('\n max_depth:', parameter_values[3])

In [ ]:
#TODO: Save the best model
model = best_run.register_model(
    model_path='outputs/hyperdrive_model.pkl',
    model_name='hyperdrive_best_model',
    tags={'Method':'Hyperdrive'},
    properties={'Accuracy': best_run_metrics['Accuracy']}
)

## Model Deployment

Remember you have to deploy only one of the two models you trained but you still need to register both the models. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

In [ ]:
env = Environment.from_conda_specification(name='sklearn_env', file_path='environment.yml')
inference_config = InferenceConfig(entry_script='score_hyperdrive.py', environment=env)

# Deploy the model as a web service
service_name = 'service-hyperdrive'
inference_config = InferenceConfig(entry_script="score_hyperdrive.py", environment=env)
aci_config = AciWebservice.deploy_configuration(cpu_cores=1, memory_gb=1)
service = Model.deploy(workspace=ws,
                       name=service_name,
                       models=[model],
                       inference_config=inference_config,
                       deployment_config=aci_config,
                       overwrite=True)
service.wait_for_deployment(show_output=True)

TODO: In the cell below, send a request to the web service you deployed to test it.

In [ ]:
scoring_uri = service.scoring_uri
data = {"data": [iris.data[0].tolist()]}
headers = {'Content-Type': 'application/json'}

response = requests.post(scoring_uri, json=data, headers=headers)
print("Prediction:", response.json())

TODO: In the cell below, print the logs of the web service and delete the service

In [ ]:
print(service.get_logs())

# Delete the web service
service.delete()

# Shutdown all the computes that have been used
# for compute_name in ws.compute_targets:
#     ws.compute_targets[compute_name].stop()

**Submission Checklist**
- I have registered the model.
- I have deployed the model with the best accuracy as a webservice.
- I have tested the webservice by sending a request to the model endpoint.
- I have deleted the webservice and shutdown all the computes that I have used.
- I have taken a screenshot showing the model endpoint as active.
- The project includes a file containing the environment details.

